In [1]:
import os
from dotenv import load_dotenv
from langchain_neo4j import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chains import RetrievalQA


import vansy_f8a_genai_test.report_a03_rag_and_reasoning_frameworks_tutorial_prompt as prompt_hub

In [2]:
load_dotenv()

True

In [3]:
memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

/var/folders/lg/yb401ry14w19vfs73n7y2rtw0000gn/T/ipykernel_66474/88119791.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


### Hospital Cypher Chain

In [4]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

graph.refresh_schema()

/var/folders/lg/yb401ry14w19vfs73n7y2rtw0000gn/T/ipykernel_66474/249957495.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [5]:
cypher_generation_template = prompt_hub.CYPHER_GENERATION_TEMPLATE
qa_generation_template = prompt_hub.QA_GENERATION_TEMPLATE

In [6]:
cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"], template=cypher_generation_template
)

qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template
)


In [7]:
hospital_cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    qa_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    graph=graph,
    verbose=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    validate_cypher=True,
    top_k=100,
    allow_dangerous_requests=True,
    # use_function_response=True
)

### Hospital Review Chain

In [8]:
from langchain_neo4j import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [9]:
neo4j_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="reviews",
    node_label="Review",
    text_node_properties=[
        "physician_name",
        "patient_name",
        "text",
        "hospital_name",
    ],
    embedding_node_property="embedding",
)

In [10]:
review_generation_template = prompt_hub.REVIEW_GENERATION_TEMPLATE

In [11]:
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_generation_template
    )
)

In [12]:
review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)

In [13]:
messages = [review_system_prompt, review_human_prompt]
review_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

In [14]:
reviews_vector_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    chain_type="stuff",
    retriever=neo4j_vector_index.as_retriever(k=12),
)
reviews_vector_chain.combine_documents_chain.llm_chain.prompt = review_prompt

### ReAct Agent

In [15]:
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_openai import ChatOpenAI
from langchain import hub

In [16]:
# 1. Define the tools for agent can use
tools = [
    Tool(
    name="HospitalGraphQA",
    # The 'func' is the chain's invoke method. We use a lambda to map the
    # string input from the agent to the dictionary input the chain expects.
    func=lambda q: hospital_cypher_chain.invoke({"query": q}),
    description="""Useful for answering questions about patients,
        physicians, hospitals, insurance payers, patient review
        statistics, and hospital visit details. Use the entire prompt as
        input to the tool. For instance, if the prompt is 'How many visits
        have there been?", the input should be "How many visits have
        there been?'
        """
    ),
    Tool(
    name="HospitalReviewsQA",
    func=lambda q: reviews_vector_chain.invoke({"query": q}),
    description="""Useful when you need to answer questions
        about patient experiences, feelings, or any other qualitative
        question that could be answered about a patient using semantic
        search. Not useful for answering objective questions that involve
        counting, percentages, aggregations, or listing facts. Use the
        entire prompt as input to the tool. For instance, if the prompt is
        "Are patients satisfied with their care?", the input should be
        "Are patients satisfied with their care?".
        """
    )
]

In [17]:
# 2. Set up the ReAct Agent
# Pull the standard ReAct prompt
prompt = hub.pull("hwchase17/react")

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
prompt.dict()

{'name': None,
 'input_variables': ['agent_scratchpad', 'input', 'tool_names', 'tools'],
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {},
 'metadata': {'lc_hub_owner': 'hwchase17',
  'lc_hub_repo': 'react',
  'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'},
 'tags': None,
 'template': 'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}',
 'template_format': 'f-string',
 'validate_t

In [19]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [20]:
custom_prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

PREVIOUS CHAT HISTORY:
{chat_history}

NEW QUESTION:
Question: {input}
Thought:{agent_scratchpad}
"""

In [21]:
prompt = PromptTemplate(
    input_variables=['chat_history','agent_scratchpad', 'input', 'tool_names', 'tools'],
    template=custom_prompt_template,
)

In [22]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [23]:
# Create the agent
agent = create_react_agent(agent_llm, tools, prompt)

In [24]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [25]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [26]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [49]:
# Create the agent executor to run the agent
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, verbose=True, 
                               memory=memory, 
                               return_intermediate_steps=True,
                               handle_parsing_errors=True)

In [32]:
# --- 3. The Interactive Chat Loop ---
print("Chat with the Hospital AI Agent! Type 'exit' to quit.")
while True:
    try:
        user_question = input("You: ")
        print(f"\nUser: {user_question}")
        if user_question.lower() in ["exit", "quit"]:
            print("\nAgent: Goodbye!")
            break
        
        # Invoke the agent executor with the user's question
        response = agent_executor.invoke({"input": user_question})
        
        # Print the final answer
        print(f"\nAgent: {response['output']}")

    except Exception as e:
        print(f"An error occurred: {e}")

Chat with the Hospital AI Agent! Type 'exit' to quit.

User: how many hospital in Florida?


> Entering new AgentExecutor chain...
I need to find out the number of hospitals in Florida. Since this is an objective question that involves counting, I will use the HospitalGraphQA tool to get the information.

Action: HospitalGraphQA  
Action Input: how many hospital in Florida?  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)
WHERE h.state_name = 'FL'
RETURN COUNT(h) AS hospital_count
Full Context:
[{'hospital_count': 7}]

> Finished chain.
{'query': 'how many hospital in Florida?', 'result': 'There are 7 hospitals in Florida.'}

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


I now know the final answer.  
Final Answer: There are 7 hospitals in Florida.

> Finished chain.

Agent: There are 7 hospitals in Florida.

User: can you name them?


> Entering new AgentExecutor chain...
I need to find out the names of the hospitals in Florida. Since this is an objective question that involves listing facts, I will use the HospitalGraphQA tool to retrieve this information.

Action: HospitalGraphQA  
Action Input: "Can you name the hospitals in Florida?"  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)
WHERE h.state_name = 'FL'
RETURN h.name AS hospital_name
Full Context:
[{'hospital_name': 'Walton LLC'}, {'hospital_name': 'Wheeler, Bryant and Johns'}, {'hospital_name': 'Brown Inc'}, {'hospital_name': 'Smith, Edwards and Obrien'}, {'hospital_name': 'Rose Inc'}, {'hospital_name': 'Jordan Inc'}, {'hospital_name': 'Lewis-Nelson'}]

> Finished chain.
{'query': 'Can you name the hospitals in Florida?', 'result': 'The hospitals in Florida a

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


I now know the final answer.  
Final Answer: The hospitals in Florida are as follows:  
1. Walton LLC  
2. Wheeler, Bryant and Johns  
3. Brown Inc  
4. Smith, Edwards and Obrien  
5. Rose Inc  
6. Jordan Inc  
7. Lewis-Nelson  

> Finished chain.

Agent: The hospitals in Florida are as follows:  
1. Walton LLC  
2. Wheeler, Bryant and Johns  
3. Brown Inc  
4. Smith, Edwards and Obrien  
5. Rose Inc  
6. Jordan Inc  
7. Lewis-Nelson

User: exit

Agent: Goodbye!


In [36]:
from langchain.callbacks import get_openai_callback


In [51]:
with get_openai_callback() as cb:
    response = agent_executor.invoke({"input": "How many hospital in CO?"})



> Entering new AgentExecutor chain...
I need to find out the number of hospitals in Colorado (CO). Since this is an objective question that involves counting, I will use the HospitalGraphQA tool to get the information.

Action: HospitalGraphQA  
Action Input: How many hospital in CO?  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)
WHERE h.state_name = 'CO'
RETURN COUNT(h) AS hospital_count
Full Context:
[{'hospital_count': 8}]

> Finished chain.
{'query': 'How many hospital in CO?', 'result': 'There are 8 hospitals in Colorado.'}I now know the final answer.  
Final Answer: There are 8 hospitals in Colorado.

> Finished chain.


/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


In [52]:
response

{'input': 'How many hospital in CO?',
 'chat_history': [HumanMessage(content='How many hospital in LA?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='There are currently no hospitals in LA.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='how many hospital in Florida?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='There are 7 hospitals in Florida.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='can you name them?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The hospitals in Florida are as follows:  \n1. Walton LLC  \n2. Wheeler, Bryant and Johns  \n3. Brown Inc  \n4. Smith, Edwards and Obrien  \n5. Rose Inc  \n6. Jordan Inc  \n7. Lewis-Nelson', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How many hospital in Florida?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='There are 7 hospitals in Florida.', additional_kwargs={}, response_met

In [53]:
cb.total_cost, cb.total_tokens

(0.00015314999999999998, 1446)

In [55]:
action = response['intermediate_steps'][0][0]
action

AgentAction(tool='HospitalGraphQA', tool_input='How many hospital in CO?', log='I need to find out the number of hospitals in Colorado (CO). Since this is an objective question that involves counting, I will use the HospitalGraphQA tool to get the information.\n\nAction: HospitalGraphQA  \nAction Input: How many hospital in CO?  ')

In [56]:
action.tool

'HospitalGraphQA'